In [10]:
# libs needed that aren't on the huggingface container 
# !pip install datasets --user
# ! pip install evaluate --user

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 16.3 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
from transformers import BertTokenizerFast, TrainingArguments, Trainer, BertForSequenceClassification, \
    DataCollatorWithPadding
from datasets import Dataset, load_dataset
import csv
import pandas as pd
from sklearn import model_selection
import torch
import evaluate
import numpy as np

In [2]:
with open('Sentiment140.csv', mode='r', encoding='latin1') as file:
    tweet_data = csv.reader(file)

    labels = ['label', 'Id', 'Date', 'Query', 'User', 'texts']
    df = pd.DataFrame(tweet_data, columns=labels)

df = df.drop(columns=['Id', 'Date', 'Query', 'User'], axis=1)
df['label'] = df['label'].replace("4", "1")
pd.to_numeric(df['label'], downcast='float')

# this is hacky & the dataset class from huggingface is supposed to be able to take in a dataframe...
# I couldn't get it to work, so I did this instead
df.to_csv("tweet_data_clean.csv", index=False, )

dataset = load_dataset('csv', data_files='tweet_data_clean.csv')
dataset = dataset['train'].train_test_split(test_size=0.2)
print(dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'texts'],
        num_rows: 1280000
    })
    test: Dataset({
        features: ['label', 'texts'],
        num_rows: 320000
    })
})


In [3]:
def tokenize_function(example):
    """
    Tokenizes input embeddings using model tokenizer"""
    return tokenizer(example["texts"], truncation=True)

def compute_metrics(eval_preds):
    """
    Computes eval metrics"""
    metric = evaluate.load("accuracy", "f1")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# tokenize input sequences to subwords
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenized_tweets = dataset.map(tokenize_function, batched=True)


# for batching
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/1280000 [00:00<?, ? examples/s]

Map:   0%|          | 0/320000 [00:00<?, ? examples/s]

In [6]:
# to load from checkpoint for eval; trainer class takes care of both training and eval.
# pretty much just a general model wrapper
model = BertForSequenceClassification.from_pretrained('results/')

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
# speeds up testing by turning off backprop objects
with torch.no_grad():
    eval_results = trainer.evaluate(eval_dataset=tokenized_tweets["test"])

/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [7]:
print(eval_results)

{'eval_loss': 0.2117958813905716, 'eval_accuracy': 0.916140625, 'eval_runtime': 666.5781, 'eval_samples_per_second': 480.064, 'eval_steps_per_second': 15.002}


In [3]:
# to train model; see https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification
import torch
print(tokenized_tweets)
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.device_count())

# init model:
# takes in a string and pulls the model from huggingface
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=30,
    per_device_eval_batch_size=30,
    num_train_epochs=3,
    weight_decay=0.01)

# train
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_tweets["train"],
    eval_dataset=tokenized_tweets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
trainer.train()
trainer.save_model()

DatasetDict({
    train: Dataset({
        features: ['label', 'texts', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1280000
    })
    test: Dataset({
        features: ['label', 'texts', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 320000
    })
})
True
cuda
4


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss
500,0.411500
1000,0.362800
1500,0.348300
2000,0.347500
2500,0.335100
3000,0.334800
3500,0.324600
4000,0.326000
4500,0.324400
5000,0.318500


/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqu

TrainOutput(global_step=32001, training_loss=0.27423925196395094, metrics={'train_runtime': 12571.3956, 'train_samples_per_second': 305.455, 'train_steps_per_second': 2.546, 'total_flos': 1.075547427655728e+17, 'train_loss': 0.27423925196395094, 'epoch': 3.0})

In [4]:
with torch.no_grad():
    eval_results = trainer.evaluate(eval_dataset=tokenized_tweets["test"])

In [5]:
print(eval_results)

{'eval_loss': 0.30886465311050415, 'eval_runtime': 320.2354, 'eval_samples_per_second': 999.265, 'eval_steps_per_second': 8.328, 'epoch': 3.0}
